# Maze solver with reinforced learning

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from collections import defaultdict
from time import time
import imageio.v2 as imageio
from IPython import display
import os
import random

In [ ]:
allowed_actions = {'D':[1,0],'U':[-1,0],'R':[0,1],'L':[0,-1]}

## Maze generator
Function takes maze shape (X,Y), objects to be generated (by default 0-normal, 1- wall) and their probability (80/20) \
It always makes points (0,0) - begining and (X,Y) - end walkable.

In [ ]:
def generate_maze(X, Y, objects, probability):
        if len(objects) != len(probability):
            print(f'[Error] Objects and probability have different number of items.')
            return None
        elif np.sum(probability) != 1.0:
            print(f'[Error] Sum of probabilities does not equal 1.0.')
            return None
        else:
            maze = np.random.choice(objects,(X,Y),p=probability)
            maze[0,0] = 0
            maze[X-1,Y-1] = 0
            key_pos_x = random.randrange(1, X-1)
            key_pos_y = random.randrange(1, Y-1)
            maze[key_pos_x,key_pos_y] = 2
            key_pos = (key_pos_x, key_pos_y)
            return maze, key_pos

## Maze class
It is responsible for doing everything thats connected directly to maze. 

### Functions find_allowed_steps, check_move, move_value,
After being initialised it generates dictionary of all possible moves. For example \
|----------------------------- \
|-Position: 0,0                 \
|   |- D -> Cost: 1             \
|   |- R -> Cost: 100           \
It is possible to print *allowed_steps* by calling **print_allowed_steps** function.

### Updating maze
After calling **update_maze** function with choosen action it:
1. Updates current robot position
   1. Also it checks if its possible to do the action. Its useful when updating maze manually.
2. It gets the *reward* of the action
3. Finally it appends the move to the *steps* list which includes the history of all steps taken by the robot.
Function returns current *robot state* and *reward*.

### Reseting maze
Calling **return_results_and_reset** returns move history (*steps*) along with *score* of the epoche. \
Then it resets *score*, *steps* and *robot position*.

### Printing maze
It is possible to call **show_maze** function to visualise current state of maze. \
Function recreates all steps taken of the robot and then prints plot of the maze.


In [ ]:
class Maze:
    def __init__(self, maze,color_scale,specials_points={0:1,1:100,2:1}):
        self.maze = maze[0]
        self.special_points = specials_points       #Dict containing points for each of the objects
        self.Y,self.X = [i-1 for i in maze[0].shape]   #Size of maze -1 because we start from (0,0)
        self.robot_position = (0,0)                 #Current position of robot
        self.steps = []                             #All steps taken by robot
        self.score = 0
        self.allowed_steps = defaultdict(list)      #List of allowed stapes
        self.find_allowed_steps()                   #Construct dict of possible steps along with cost
        self.color_scale = color_scale
        self.has_key = False
        self.key_pos_c = tuple(maze[1])

    def check_move(self, y, x, action):
        y += allowed_actions[action][0]
        x += allowed_actions[action][1]
        if x < 0 or y < 0 or x > self.X or y > self.Y:
            return False
        else:
            return True

    def move_value(self, y, x, action):
        y += allowed_actions[action][0]
        x += allowed_actions[action][1]
        return self.special_points[self.maze[y,x]]

    def check_game_over(self):
        if self.robot_position == (self.Y,self.X) and self.has_key == True:
            return True
        else:
            return False
    
    def find_allowed_steps(self):
        for y in range(self.Y+1):
            for x in range(self.X+1):
                for action in allowed_actions:
                    if self.check_move(y,x,action):
                        self.allowed_steps[(y,x)].append(action)
    
    def print_allowed_steps(self):
        string_size = 30
        print('|' + '-'*(string_size-1))
        for k, v in self.allowed_steps.items():
            string = f'Position: {k[0]},{k[1]}'
            for action in v:
                string += '\n'
                string += f'|   |- {action} -> Cost: {self.move_value(k[0],k[1],action)}'
            print('|-' + string)
        print('|' + '-'*(string_size-1))
        return self.allowed_steps

    def update_maze(self,action):
        y, x = self.robot_position
        if self.check_move(y, x, action):
            reward = self.move_value(y,x,action)
            self.score += reward
            y += allowed_actions[action][0]
            x += allowed_actions[action][1]
            self.robot_position = (y,x)
            self.steps.append(action)
            #print(self.score)
            if self.robot_position == self.key_pos_c:
                self.has_key = True
        else:
            print('Move imposible.')
        return self.robot_position, reward
    
    def return_results_and_reset(self):
        result_score = self.score
        result_steps = self.steps
        self.score = 0
        self.steps = []
        self.robot_position = (0,0)
        self.has_key = False
        return result_score, result_steps
    
    
    def show_maze(self):
        cmap = colors.ListedColormap(self.color_scale)
        bounds = np.arange(len(self.color_scale)+1)
        norm = colors.BoundaryNorm(bounds,cmap.N)
        maze_copy = self.maze.copy()
        maze_copy_copy = self.maze.copy()
        y, x = (0,0)
        for action in self.steps:
            maze_copy[(y,x)] = maze_copy_copy[(y,x)] + 5
            y += allowed_actions[action][0]
            x += allowed_actions[action][1]
            maze_copy[(y,x)] = len(self.color_scale)-1
        plt.figure(figsize=(X,Y))
        plt.figure()
        im = plt.imshow(maze_copy,interpolation='none', aspect='equal',
                        cmap=cmap,norm=norm)
        ax = plt.gca()
        # Major ticks
        ax.set_xticks(np.arange(0, self.X+1, 1))
        ax.set_yticks(np.arange(0, self.Y+1, 1))
        # # Minor ticks
        ax.set_xticks(np.arange(-.5, self.X+1, 1), minor=True)
        ax.set_yticks(np.arange(-.5, self.Y+1, 1), minor=True)
        # Gridlines based on minor ticks
        ax.grid(which='minor', color='black', linestyle='-', linewidth=1)
        # Remove minor ticks
        ax.tick_params(which='minor', bottom=False, left=False)

## Agent class
Class responsible for learning robot to solve the maze.\
The goal of the robot is to minimize *reward*, so be as close to *target* (0) as possible.

### Expected reward
The agent starts with calling **init_reward** function which generates table (*G*) of random expected rewards for each move. The reward are between 0.1 to max(special_points).

### Loss function
After each epoche agent updates expected reward for each state using this formula:
$$ G_{state} = G_{state} + \alpha(target-G_{state})$$
Where $\alpha$ is learning rate and $target$ is the target reward. So this formula adds some percantage of difference between actual reward and expected reward.

### Learning

Function **learn** is responsible for updating the values of *G* (basically learning). It knows the target (because we minimize we chose 0)  and updates values using formula above. 

### Choosing action
To choose an action agent firstly decides between exploring and exploiting the maze. \
Exploring makes agent go to random directions to explore rewards in the maze. \
Exploiting means that the agent will choose an action where reward will be the lowest. \
We set the algorythm to explore 20% of the time but we will decrease this number over time as the agent learns rewards in the maze.

In [ ]:
class Agent:
    def __init__(self, states,special_points, alpha=0.15, random_factor=0.2):
        self.special_points = special_points
        self.state_history = [[(0,0),0]]
        self.alpha = alpha
        self.random_factor = random_factor
        self.G = {}
        self.init_rewards(states)
        
    def init_rewards(self, states):
        #Initialize random state table 
        for i, row in enumerate(states):
            for j, col in enumerate(row):
                self.G[(i,j)] = np.random.uniform(high=10.0, low=0.1)

    def update_state_history(self, state, reward):
        self.state_history.append((state, reward))
    
    def learn(self):
        target = 0                                                          # we know the "ideal" reward
        
        for state, reward in reversed(self.state_history):
            self.G[state] = self.G[state]+ self.alpha * (target - self.G[state])
            target -= reward
        self.state_history = []                                             # reset the state_history
        self.random_factor -= 10e-6                                      # decrease random_factor


    def choose_action(self, state, allowed_moves):
        maxG = -10e15
        next_move = None
        if np.random.random() < self.random_factor:
            next_move = np.random.choice(allowed_moves)
        else:                                       
            for action in allowed_moves:
                new_state = tuple([sum(x) for x in zip(state, allowed_actions[action])])
                if self.G[new_state] >= maxG:
                    next_move = action
                    maxG = self.G[new_state]
        return next_move

In [ ]:
def print_epoche_results(i, t, score, steps_taken, best_score, best_epoch):
    #Funtion that prints results of each epoch
    string_len = 20
    epoche_string = f'Epoche {i}'
    epoche_string = epoche_string + " "*(string_len-len(epoche_string)-1) + ' | '

    score_string = f'Score: {score}'
    score_string = score_string + " "*(string_len-len(score_string)-1) + ' | '

    steps_string = f'Steps: {len(steps_taken)}'
    steps_string = steps_string + " "*(string_len-len(steps_string)-1) + ' | '

    time_string = f'Time: {np.round(t,4)}'
    time_string = time_string + " "*(string_len-len(time_string)-1) + ' | '

    best_score_string = f'Best score: {best_score}'
    best_score_string = best_score_string + " "*(string_len-len(best_score_string)-1) + ' | '

    best_epoch_string = f'Best epoch: {best_epoch}'
    best_epoch_string = best_epoch_string + " "*(string_len-len(best_epoch_string)-1) + ' | '

    print('| '+epoche_string+score_string+steps_string+time_string+best_score_string+best_epoch_string)

In [ ]:
def check_stop_condition(best_result, best_epoch, score, epoch, stop_after = 200):
    #Stop learning after given number of iterations without better result
    
    if best_result > score:
        best_result = score
        best_epoch = epoch
        return False, best_result, best_epoch
    elif epoch - best_epoch == stop_after:
        return True, best_result, best_epoch
    else:
        return False, best_result, best_epoch

In [ ]:
def recreate_game(steps,maze,color_scale,tick_labels):
    if not os.path.exists('img'):
        os.makedirs('img')
    filenames = []
    maze_copy = maze.copy()
    maze_copy_copy = maze.copy()
    cmap = colors.ListedColormap(color_scale)
    bounds = np.arange(len(color_scale)+1)
    norm = colors.BoundaryNorm(bounds,cmap.N)
    Y, X = maze.shape
    y, x = (0,0)
    for index, action in enumerate(steps):
        maze_copy[(y,x)] = maze_copy_copy[(y,x)] + 5
        y += allowed_actions[action][0]
        x += allowed_actions[action][1]
        maze_copy[(y,x)] = len(color_scale)-1
        plt.figure(figsize=(X,Y))
        im = plt.imshow(maze_copy,interpolation='none', aspect='equal',
                        cmap=cmap,norm=norm)
        ax = plt.gca()
        # Major ticks
        ax.set_xticks(np.arange(0, X, 1))
        ax.set_yticks(np.arange(0, Y, 1))
        # # Minor ticks
        ax.set_xticks(np.arange(-.5, X, 1), minor=True)
        ax.set_yticks(np.arange(-.5, Y, 1), minor=True)
        # Gridlines based on minor ticks
        ax.grid(which='minor', color='black', linestyle='-', linewidth=1)
        # Remove minor ticks
        ax.tick_params(which='minor', bottom=False, left=False)
        #Colorbar
        cbar = plt.colorbar(im,shrink=0.75)
        cbar.set_ticks(bounds)
        cbar.set_ticklabels(tick_labels)
        filenames.append(f'{index}.png')
        plt.savefig(os.path.join('img',f'{index}.png'))
    return filenames

In [ ]:
def generate_gif(filenames,name='result.gif', frame_duration = 0.1):
    with imageio.get_writer(name, mode='I', duration = frame_duration) as writer:
        for filename in filenames:
            image = imageio.imread(os.path.join('img',filename))
            writer.append_data(image)

    for filename in filenames:
        os.remove(os.path.join('img',filename)) 

In [ ]:
#X, Y = 10, 10
X, Y = 15, 15
points_path = 2
points_wall = 100
points_boost = 1
points_slowdown = 10
key = 2
special_points = {0:points_path,1:points_wall, 2:key, 3:points_boost,4:points_slowdown, 5:points_path, 6:points_wall, 7:key, 8:points_boost, 9:points_slowdown}
prob = [0.65,0.15,0,0.1,0.1,0,0,0,0,0]
color_scale = ['white','red','palegreen','lightblue','orange','grey','darkred','darkgreen','teal','brown','black']
tick_labels = ["","Path", "Wall", 'Key', "Boost", "Slowdown", 'Walked\npath','Walked\nwall', 'Walked\nKey', 'Walked\nboost', 'Walked\nslowdown', 'Current\nposition']
generated_maze = generate_maze(X,Y,list(special_points.keys()),prob)

In [ ]:
generated_maze = generate_maze(X,Y,list(special_points.keys()),prob)

In [ ]:
maze = Maze(generated_maze, color_scale,special_points)
robot = Agent(maze.maze,special_points, alpha=0.0001, random_factor=0.2)
maze.show_maze()

## Main loop

In [ ]:
epoches = 100000
best_result = 0
best_epoch = epoches
results = {}
overall_time = time()
for i in range(epoches):
    print('| '+('-'*20+'| ')*6)
    t = time()
    while not maze.check_game_over():    
        state = maze.robot_position        
        action = robot.choose_action(state, maze.allowed_steps[state])      # Choose an action (explore or exploit)
        state, reward = maze.update_maze(action)                            # Update maze state
        robot.update_state_history(state, reward)                           # Update robot memory and reward
        if len(maze.steps) > (X*Y*2):                                       # If robot cannot reach the end in given time move it to the end
            maze.robot_position = (maze.Y, maze.X)
            maze.has_key = True
    robot.learn()                                                          
    score, steps_taken = maze.return_results_and_reset()                    # Return result and reset maze
    if best_result == 0:                                                    # Check stop condition
        best_result = score
        best_epoch = i
    else:
        stop, best_result, best_epoch = check_stop_condition(best_result, best_epoch, score, i,epoches//5)
        if stop:
            print(f'Learning stopped. The best score has not been improved for {i-best_epoch} epochs.')
            break
    results[score] = steps_taken                                            # Update best scores
    print_epoche_results(i, time()-t, score,steps_taken, best_result, best_epoch)                    # Print epoche results
print(f'Learning was done in {np.round(time()-overall_time,1)} seconds')

In [ ]:
best_steps = results[min(results.keys())]
print(f'Best score is {best_result} achieved in {len(results[best_result])} steps in epoche {best_epoch}')
print(f'Steps taken to win:')
for step in best_steps:
    print(f'->',end=' ')
    print(f'{step}',end=' ')
print('')

In [ ]:
name = 'result.gif'
filenames = recreate_game(best_steps,maze.maze,color_scale,tick_labels)

In [ ]:
generate_gif(filenames,name,0.3)

In [ ]:
display.Image(data=open(name,'rb').read(), format='png')